# Versionnage et traçage des données

Nous allons étudier dans ces travaux pratiques comment versionner ses données et conserver des informations d'origine à l'aide de l'outil DVC.

Cet outil découple le versionnage des données et leur stockage.

## Notes sur l'environnement CodeSpace

Vous pouvez exécuter des commandes dans le terminal directement ou dans les cellules de code de ce notebook. Dans ce cas, elles doivent préfixées d'un `!`, elles seraient sinon interprétées comme du Python.

Par ailleurs, si vous avez besoin de copier-coller quelque chose dans le terminal, il faut utiliser ctrl-maj-v et non pas ctrl-v.

## Création du dépôt DagsHub

Créez un compte sur [GitHub](https://github.com) et [DagsHub](https://dagshub.com/) si nécessaire, puis&nbsp;:

Créez ensuite un dépôt vierge sur DagsHub. Une fois le dépôt créé, récupérez votre clef d'accès secrète en cliquant sur «&nbsp;*Get started with Data*&nbsp;» et en la copiant-collant depuis la section «&nbsp;*Connection credentials*&nbsp;».

Remplissez les variables `owner`, `repo` & `token` avec respectivement votre nom d'utilisateur DagsHub, votre nom de dépôt git et votre clef d'accès secrète.

In [8]:
owner = "BGuegan"
repo = "my-first-repo"
token = "fba50dc25afd8c819c9c66127576a8313ce524af"

Exécutez maintenant la cellule ci-dessous pour créer une branche distante pointant vers votre dépôt DagsHub (ce qui nous permettra de publier les changements produits pendant ces travaux pratiques pour simuler une tâche réelle).

In [7]:
!git remote add dagshub https://{token}@dagshub.com/{owner}/{repo}.git

error: remote dagshub already exists.


## Initialisation du dépôt DVC

Dans le terminal, lancez une commande pour initialiser un dépôt DVC puis publiez les changements sur votre dépôt DagsHub.

### Solution

In [5]:
!dvc init

ERROR: failed to initiate DVC - '.dvc' exists. Use `-f` to force.


In [15]:
!git commit -m "Initialisation de DVC"

On branch main
Your branch is ahead of 'dagshub/main' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   "Versionnage_et_tra\303\247age_des_donn\303\251es.ipynb"

no changes added to commit (use "git add" and/or "git commit -a")


In [25]:
!git push -u dagshub main

branch 'main' set up to track 'dagshub/main'.
Everything up-to-date


## Récupération d'une première version des données

Il y a plusieurs manières d'importer une source de données dans DVC. L'une d'entre elles est d'utiliser deux commandes&nbsp;: [`dvc get`](https://dvc.org/doc/command-reference/get) d'abord, qui récupère des données depuis un dépôt git/DVC, puis [`dvc add`](https://dvc.org/doc/command-reference/add) qui ajoute les données aux données gérées par DVC.

Après avoir consulté leur aide, utilisez ces deux commandes pour importer le fichier `wiki_movie_plots_deduped.csv` du dépôt git `https://github.com/m09/dataset-wikipedia-movie-plots/` sous le nom `data.csv`.

DVC utilise un dossier de cache qui contient (la plupart du temps) les données utilisées par le projet. Que constatez-vous en observant ce dossier&nbsp;?

N'hésitez pas à observer les fichiers d'extension `.dvc`.

### Solution

In [10]:
!dvc get https://github.com/m09/dataset-wikipedia-movie-plots wiki_movie_plots_deduped.csv -o data.csv

In [11]:
!dvc add data.csv

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/workspaces/dvc-playground/.dvc/cache/files/md5'| |0/? [
                                                                                
!
  0%|          |Adding data.csv to cache              0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /workspaces/dvc-playgroun0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00,  1.98file/s]

To track the changes with git, run:

	git add data.csv.dvc .gitignore

To enable auto staging, run:

	dvc config core.autostage true


## Ajout du fichier à un commit git

Commitez maintenant ce fichier en suivant les recommandations de DVC données en sortie de la commande `dvc add`, puis publiez ces changements sur DagsHub.

### Solution

In [21]:
!git add .gitignore data.csv.dvc

In [26]:
!git commit -m "Ajout d'une première version des données"

On branch main
Your branch is up to date with 'dagshub/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   "Versionnage_et_tra\303\247age_des_donn\303\251es.ipynb"

no changes added to commit (use "git add" and/or "git commit -a")


In [23]:
!git tag "v1"

fatal: tag 'v1' already exists


In [24]:
!git push origin main v1

remote: Permission to shuuchuu/dvc-playground.git denied to BGuegan.
fatal: unable to access 'https://github.com/shuuchuu/dvc-playground/': The requested URL returned error: 403


## Configuration d'un serveur de stockage

DVC peut stocker les données dans divers types de serveurs. DagsHub met à disposition un espace de stockage qui s'utilise comme un bucket s3 (solution de stockage du cloud amazon).

Configurez ce serveur en suivant les instructions disponibles dans l'onglet « *Data* » du bouton vert « *Remote* » de votre dépôt DagsHub puis publiez-y vos données.

### Solution

In [27]:
!dvc remote add origin s3://dvc
!dvc remote modify origin endpointurl https://dagshub.com/{owner}/{repo}.s3
!dvc remote modify origin --local access_key_id {token}
!dvc remote modify origin --local secret_access_key {token}

In [28]:
!dvc push -r origin

Pushing
!
  0% Checking cache in 'dvc/files/md5'|              |0/? [00:00<?,    ?files/s]
                                                                                
!
  0% Checking cache in '/workspaces/dvc-playground/.dvc/cache/files/md5'| |0/? [
                                                                                
!
  0%|          |Pushing to s3                         0/1 [00:00<?,     ?file/s]
  0%|          |Pushing to s3                         0/1 [00:00<?,     ?file/s]

!

  0%|          |/workspaces/dvc-playground/.dvc0.00/77.4M [00:00<?,        ?B/s]

100%|██████████|/workspaces/dvc-playground77.4M/77.4M [00:04<00:00,    19.6MB/s]

                                                                                
100%|██████████|Pushing to s3                     1/1 [00:04<00:00,  4.15s/file]
Pushing                                                                         
1 file pushed


## Modification des données

Exécutez les cellules suivantes pour modifier le fichier de données&nbsp;:

In [29]:
!sort -r < data.csv > a && dvc remove data.csv.dvc && mv a data.csv

Utilisez maintenant `dvc add` & `git add` pour enregistrer ce changement.

### Solution

In [30]:
!dvc add data.csv

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/workspaces/dvc-playground/.dvc/cache/files/md5'| |0/? [
                                                                                
!
  0%|          |Adding data.csv to cache              0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /workspaces/dvc-playgroun0/1 [00:00<?,    ?files/s]
100%|██████████|Checking out /workspaces/dvc-play1/1 [00:00<00:00,  8.42files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00,  2.84file/s]

To track the changes with git, run:

	git add .gitignore data.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true


In [36]:
!git add data.csv.dvc .gitignore

In [37]:
!git commit -m "Données v2"
!git tag "v2"

On branch main
Your branch is ahead of 'dagshub/main' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .dvc/config
	modified:   "Versionnage_et_tra\303\247age_des_donn\303\251es.ipynb"

no changes added to commit (use "git add" and/or "git commit -a")


fatal: tag 'v2' already exists


In [39]:
!git push origin main v2

remote: Permission to shuuchuu/dvc-playground.git denied to BGuegan.
fatal: unable to access 'https://github.com/shuuchuu/dvc-playground/': The requested URL returned error: 403


In [35]:
!dvc push -r origin

Pushing
!
  0% Checking cache in 'dvc/files/md5'|              |0/? [00:00<?,    ?files/s]
                                                                                
!
  0% Checking cache in '/workspaces/dvc-playground/.dvc/cache/files/md5'| |0/? [
                                                                                
!
  0%|          |Pushing to s3                         0/1 [00:00<?,     ?file/s]
  0%|          |Pushing to s3                         0/1 [00:00<?,     ?file/s]

!

  0%|          |/workspaces/dvc-playground/.dvc0.00/77.4M [00:00<?,        ?B/s]

100%|██████████|/workspaces/dvc-playground77.4M/77.4M [00:07<00:00,    11.4MB/s]

                                                                                
100%|██████████|Pushing to s3                     1/1 [00:07<00:00,  7.10s/file]
Pushing                                                                         
1 file pushed


## Retour aux données originales

On imagine avoir détecté un problème avec nos nouvelles données&nbsp;: on souhaite revenir à la première version. Utilisez `git checkout` pour revenir à la version antérieure des métadonnées, puis `dvc checkout` pour récupérer le fichier correspondant.

Commitez maintenant ce retour à la première version.

### Solution

In [ ]:
!git checkout v1 data.csv.dvc

In [40]:
!dvc checkout data.csv

Building workspace index                              |1.00 [00:00,  118entry/s]
Comparing indexes                                    |2.00 [00:00, 1.68kentry/s]
Applying changes                                      |1.00 [00:00,  9.90file/s]
M       data.csv


In [41]:
!git add data.csv.dvc

In [ ]:
!git commit -m "Retour aux données v1"
!git tag v3

You don't have write access to the shuuchuu/dvc-playground repository, so you cannot push changes to it.
To obtain write access we will point this codespace at your fork of shuuchuu/dvc-playground, creating that fork if it doesn't exist.

Would you like to proceed? 

In [ ]:
!git push origin main v3

In [ ]:
!dvc push -r origin

## Traitement des données

Exécutez la cellule suivante qui contient un script Python qui prend deux arguments, et qui écrit au chemin donné par le deuxième argument le contenu du fichier au chemin donné par le premier argument, en passant le contenu en majuscule.

Cette cellule va écrire ce script au chemin `upper.py`

In [1]:
%%writefile upper.py
from pathlib import Path
from sys import argv

Path(argv[2]).write_text(
    Path(argv[1]).read_text(encoding="utf8").upper(),
    encoding="utf8")

Writing upper.py


Ajoutez une étape de traitement avec [`dvc stage add`](https://dvc.org/doc/command-reference/stage/add) qui prend en entrée le fichier `data.csv` et produit le fichier `data-upper.csv` à partir de ce script.

Exécutez maintenant cette étape de traitement avec [`dvc repro`](https://dvc.org/doc/command-reference/repro) puis sauvegardez les données dans le serveur de stockage et les métadonnées dans git.

### Solution

In [ ]:
!dvc stage add -n transform-uppercase -d data.csv -o data-upper.csv python upper.py data.csv data-upper.csv

In [ ]:
!dvc repro

In [ ]:
!git add dvc.yaml .gitignore dvc.lock

In [ ]:
!git commit -m "Pipeline pour passer un fichier en majuscule"

In [ ]:
!git push origin main

In [ ]:
!dvc push -r origin